# A Cando-app to setup free energy perturbation calculations using AMBER

First we do the following:

  * Load the TIRUN package.
  * Set things up to carry out geometry optimization using Amber.

In [ ]:
startSwank 4011

In [ ]:
quickload :tirun-jupyter

In [ ]:
(leap:setup-amber-paths)

# Start a new TIRUN calculation

In [ ]:
(tirun-jupyter:new-tirun)

(define-symbol-macro all-ligands (tirun-jupyter::all-ligands tirun-jupyter::*app*))
(define-symbol-macro selected-ligands (tirun-jupyter::selected-ligands tirun-jupyter::*app*))
(define-symbol-macro tiruns tirun-jupyter::*tirun*)

# Load the Thrombin PDB file

In [ ]:
(tirun-jupyter:load-receptor)

In [ ]:
(apropos "all-ligands")

In [ ]:
thrombin = (load-pdb "2zff_fixed.pdb" :ignore-missing-topology t)

In [ ]:
(build-unbuilt-hydrogens thrombin)

In [ ]:
show thrombin

In [ ]:
(remove-molecules thrombin 'solvent)

In [ ]:
(chem:setf-molecule-type (chem:content-with-name thrombin :|53U_3|) 'ligand)

In [ ]:
ligmol = (chem:content-with-name thrombin :|53U_3|)

In [ ]:
(chem:remove-molecule thrombin ligmol)

In [ ]:
show thrombin

In [ ]:
show ligmol

In [ ]:
(setf (tirun:receptors tiruns) (list thrombin))

In [ ]:
(setf (tirun::template-ligands tiruns) (list ligmol))

# Sketch the ligands using Kekule composer

In [ ]:
(tirun-jupyter::load-app)

In [ ]:
(tirun-jupyter:composer)

In [ ]:
(tirun::pose-ligands-using-similarity tiruns ligmol)

In [ ]:
all-ligands

In [ ]:
selected-ligands

In [ ]:
(tirun::ligands tiruns)

# Import the sketch from a Chemdraw file

In [ ]:
(jupyter:png-file "ligand3.png" )

In [ ]:
sketch = (tirun:load-chem-draw-tirun "ligand3.cdxml")

In [ ]:
(tirun::assemble-ligands tiruns sketch)

In [ ]:
(tirun::pose-ligands-using-similarity tiruns ligmol)

In [ ]:
all-ligands = (mapcar #'tirun::molecule (tirun::ligands tiruns))

In [ ]:
selected-ligands = all-ligands

# The two approaches rejoin here.  The ligands are in tiruns and in all-ligands and selected-ligands

In [ ]:
(length all-ligands)

In [ ]:
(tirun-jupyter:select-ligands)

In [ ]:
selected-ligands

In [ ]:
(length selected-ligands)

# Define the pairs of compounds between which we want to carry out free energy perturbation calculations

In [ ]:
(tirun-jupyter:lomap)

In [ ]:
(tirun-jupyter:configure-jobs)

In [ ]:
(tirun-jupyter:write-jobs)